## Generalized lomb scargle periodogram

The [Lomb](http://adsabs.harvard.edu/abs/1976Ap%26SS..39..447L)-[Scargle](http://adsabs.harvard.edu/abs/1982ApJ...263..835S) periodogram performs a least-squares fit of sinusoids to your data

$$
P_{LS} = \frac{1}{2}(\chi^2_0 - \chi^2(\omega))
$$

where

$$
\chi^2(\omega) = \min_{\theta}\left[\sum_i w_i (y_i - \hat{y}(t_i, \omega|\theta))^2\right],
$$

is the sum of squared residuals for the best fit model,

$$
\chi^2_0 = \sum_i w_i (y_i - \bar{y})^2,
$$

is the sum of squared residuals for a constant model, with

$$
w_i = W \sigma_i^{-2},
$$

$$
W = \left[\sum \sigma_i^{-2}\right]^{-1},
$$

being the weights for the weighted sum, and

$$
\bar{y} = \sum w_i y_i
$$

being the weighted mean of the data. The model being fit to the data takes the form

$$
\hat{y}(t_i, \omega |\theta) = \theta_1\cos{\omega t_i} + \theta_2\sin{\omega t_i} + \theta_3.
$$

The original Lomb-Scargle implementation omits the constant offset -- $\theta_3$ -- parameter, which was introduced in [Zechmeister and Kurster (2009)](http://adsabs.harvard.edu/abs/2009A%26A...496..577Z) 

If you want to learn more about the Lomb-Scargle periodogram, its implications, limitations, and extensions, see [Vanderplas (2017)](https://arxiv.org/abs/1703.09824). 

The [astropy](http://docs.astropy.org/en/stable/index.html) package has an implementation with instructive [documentation](http://docs.astropy.org/en/stable/stats/lombscargle.html)

The cuvarbase lomb scargle implementation is normalized to


In [ ]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle
import astropy.units as u
from astropy.table import Table
import pandas as pd

def filter_1_sigma(tab, col):
    hi = (np.mean(tab[col]) + np.std(tab[col]))
    lo = (np.mean(tab[col]) - np.std(tab[col]))
    tab = tab[tab[col] < hi]
    tab = tab[tab[col] > lo]
    return tab, lo, hi


all_dicts = []
for path in glob('../lightcurves/segments/*.fits'):
    d = {}
    simbad_name, lc_name, MJD_START, MJD_END = path.split('/')[-1][:-5].split(',')
    
    tab = Table.read(path)

    if 'UVOT' in path:
        t, y, dy = tab['MJD'], tab['MAG'], tab['MAG_ERR']
        #tab, lo, high = filter_1_sigma(tab, 'MAG')
    elif 'XRT' in path:
        t, y, dy = tab['MJD'], tab['RATE'], tab['RATE_ERR']
        #tab, lo, high = filter_1_sigma(tab, 'RATE')

    
    if len(tab) < 30:
        continue
    
    # Run lomb scargle
    t.unit = u.second
    ls = LombScargle(t, y, dy)
    freqs, powers = ls.autopower(minimum_frequency=0.01*u.Hz) #

    f0 = freqs[np.argmax(powers)].value
    fap = ls.false_alarm_probability(powers.max())

    
    d['simbad_name'] = simbad_name
    d['lc_name'] = lc_name
    d['MJD_START'] = MJD_START
    d['MJD_END'] = MJD_END
    d['f0'] = f0
    d['t0'] = 1/f0
    d['fap'] = fap
    all_dicts.append(d)
    
    
    # Plot
    plt.figure(figsize=(10,5))
    plt.suptitle(path)
    ax1 = plt.subplot(211)
    ax1.errorbar(t, y, yerr=dy, lw=1.0, capsize=1.0)
    #ax1.axhline(hi, ls='dotted', lw=1.0)
    #ax1.axhline(lo, ls='dotted', lw=1.0)

    ax1.set_ylabel('Mag.')
    ax1.set_xlabel('MJD')

    ax2 = plt.subplot(223)
    ax2.scatter((t * f0) % 1.0, y, c='k', s=1)
    ax2.set_xlabel('Phase')
    ax2.set_ylabel('Mag.')

    ax3 = plt.subplot(224)
    ax3.plot(freqs, powers)
    ax3.axvline(f0, ls=':', color='r', label=f'$f_0=${f0:.2f} ({1/f0:.2f} days) FAP={fap*100:.2f}%')
    ax3.set_xlabel('Freq.')
    ax3.set_ylabel('$P_{LS}(f)$')
    ax3.legend()
    
    plt.savefig(f'../figures/lomb_scargle/{simbad_name},{lc_name},{MJD_START},{MJD_END}.png')
    plt.show()
    
df = pd.DataFrame(all_dicts)
print(df)

In [ ]:
pd.set_option('display.max_rows', 500)
df.sort_values('fap')


## Using `LombScargleAsyncProcess`

``cuvarbase`` uses its own implementation of the [non-equispaced fast Fourier transform](http://epubs.siam.org/doi/abs/10.1137/0914081) to compute frequency-dependent sums in the calculation of the periodogram. Using the NFFT to speed up the calculation of the LS periodogram was first demonstrated in [Leroy (2012)](https://www.aanda.org/articles/aa/abs/2012/09/aa19076-12/aa19076-12.html).


# Notes

## Precision
* `use_double`: 
* `m`
* `sigma`

## Choosing frequencies


## Running LS on many lightcurves